In [1]:
import pandas as pd
import random

def asignar_tutores(disponibilidad):
    """
    Asigna tutores a bloques de 2 horas consecutivas minimizando choques de horario.

    Args:
        disponibilidad: DataFrame de pandas con la disponibilidad de los mentores

    Returns:
        tuple: (asignacion_final, choques) donde asignacion_final es un diccionario
               {mentor: [horarios]} y choques es el número total de choques
    """
    # Convertir el DataFrame a un diccionario para mejor manejo
    mentores = disponibilidad.set_index('MentorID').to_dict('index')

    # Paso 1: Generar solución inicial aleatoria
    asignacion_actual = {}
    for mentor in mentores:
        slots_disponibles = [s for s, disp in mentores[mentor].items() if disp == 1]
        # Buscar bloques de 2 horas consecutivas disponibles
        bloques = []
        for i in range(len(slots_disponibles)-1):
            if int(slots_disponibles[i+1][4:]) == int(slots_disponibles[i][4:]) + 1:
                bloques.append((slots_disponibles[i], slots_disponibles[i+1]))
        if bloques:
            bloque_elegido = random.choice(bloques)
            asignacion_actual[mentor] = list(bloque_elegido)
        else:
            # Si no hay bloques de 2 horas, asignar cualquier horario disponible (aunque no sea consecutivo)
            if slots_disponibles:
                asignacion_actual[mentor] = [random.choice(slots_disponibles)]
            else:
                asignacion_actual[mentor] = []

    # Función para calcular choques
    def calcular_choques(asignacion):
        contador = {}
        for mentor, slots in asignacion.items():
            for slot in slots:
                contador[slot] = contador.get(slot, 0) + 1
        return sum(max(0, count-1) for count in contador.values())

    # Paso 2: Búsqueda local
    max_iter = 1000
    for _ in range(max_iter):
        choques_actual = calcular_choques(asignacion_actual)
        if choques_actual == 0:
            break  # Solución óptima encontrada

        # Generar vecino: cambiar horario de un mentor aleatorio
        mentor_cambiar = random.choice(list(asignacion_actual.keys()))
        slots_disponibles = [s for s, disp in mentores[mentor_cambiar].items() if disp == 1]

        # Buscar bloques de 2 horas consecutivas disponibles para este mentor
        bloques = []
        for i in range(len(slots_disponibles)-1):
            if int(slots_disponibles[i+1][4:]) == int(slots_disponibles[i][4:]) + 1:
                bloques.append((slots_disponibles[i], slots_disponibles[i+1]))

        # Crear nueva asignación de prueba
        nueva_asignacion = asignacion_actual.copy()
        if bloques:
            nuevo_bloque = random.choice(bloques)
            nueva_asignacion[mentor_cambiar] = list(nuevo_bloque)
        elif slots_disponibles:
            nueva_asignacion[mentor_cambiar] = [random.choice(slots_disponibles)]
        else:
            nueva_asignacion[mentor_cambiar] = []

        # Evaluar si el vecino es mejor
        nuevos_choques = calcular_choques(nueva_asignacion)
        if nuevos_choques <= choques_actual:
            asignacion_actual = nueva_asignacion

    return asignacion_actual, calcular_choques(asignacion_actual)

# Ejemplo de uso:
# Cargar datos desde CSV
disponibilidad = pd.read_csv('mentorAvailability.csv')
asignacion, choques = asignar_tutores(disponibilidad)
print("Asignación final:", asignacion)
print("Número de choques:", choques)

Asignación final: {'M01': ['Slot9', 'Slot10'], 'M02': ['Slot1', 'Slot2'], 'M03': ['Slot4', 'Slot5'], 'M04': ['Slot7', 'Slot8'], 'M05': ['Slot1', 'Slot2'], 'M06': ['Slot5', 'Slot6'], 'M07': ['Slot6', 'Slot7'], 'M08': ['Slot1', 'Slot2'], 'M09': ['Slot5', 'Slot6'], 'M10': ['Slot3', 'Slot4'], 'M11': ['Slot1'], 'M12': ['Slot1', 'Slot2'], 'M13': ['Slot4', 'Slot5'], 'M14': ['Slot7', 'Slot8'], 'M15': ['Slot5', 'Slot6'], 'M16': ['Slot1', 'Slot2'], 'M17': ['Slot2', 'Slot3'], 'M18': ['Slot6', 'Slot7'], 'M19': ['Slot8', 'Slot9'], 'M20': ['Slot7']}
Número de choques: 28
